In [22]:
# # enanle print everywhere
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all" 

In [23]:
# Module name, and data content
name = "lol"
input_json = {"name": "hi", "age": 1}
name = name.lower()
entity_name = name.capitalize()
# Json

# Generates module, controller, service

In [40]:
# if next cell print nest: command not found, execute this
# !npm install -g @nestjs/cli

npm WARN deprecated urix@0.1.0: Please see https://github.com/lydell/urix#deprecated
npm WARN deprecated resolve-url@0.2.1: https://github.com/lydell/resolve-url#deprecated
npm WARN deprecated chokidar@2.1.8: Chokidar 2 will break on node v14+. Upgrade to chokidar 3 with 15x less dependencies.
npm WARN deprecated fsevents@1.2.12: fsevents 1 will break on node v14+ and could be using insecure binaries. Upgrade to fsevents 2.
/Users/stevensheng/.nvm/versions/node/v12.16.1/bin/nest -> /Users/stevensheng/.nvm/versions/node/v12.16.1/lib/node_modules/@nestjs/cli/bin/nest.js
7m            ░░░░░░⸩ ⠹ install:mkdirp: info lifecycle mkdirp@0.5.5~install: mkd
> fsevents@1.2.12 install /Users/stevensheng/.nvm/versions/node/v12.16.1/lib/node_modules/@nestjs/cli/node_modules/watchpack/node_modules/fsevents
> node-gyp rebuild

  SOLINK_MODULE(target) Release/.node
  CXX(target) Release/obj.target/fse/fsevents.o
  SOLINK_MODULE(target) Release/fse.node
+ @nestjs/cli@7.1.5░░░░░⸩ ⠇ postinstall: info life

In [41]:
! nest g module ./modules/$name
! nest g controller ./modules/$name
! nest g service ./modules/$name

CREATE src/modules/lol/lol.module.ts (80 bytes)
UPDATE src/app.module.ts (1789 bytes)
CREATE src/modules/lol/lol.controller.spec.ts (472 bytes)
CREATE src/modules/lol/lol.controller.ts (95 bytes)
UPDATE src/modules/lol/lol.module.ts (162 bytes)
CREATE src/modules/lol/lol.service.spec.ts (439 bytes)
CREATE src/modules/lol/lol.service.ts (87 bytes)
UPDATE src/modules/lol/lol.module.ts (233 bytes)


#  Create entity file

In [42]:
import json
with open(f'{name}.json', 'w') as file:  # Use file to refer to the file object
    json.dump(input_json, file)

In [43]:
with open(f'{name}.json', 'r') as file:  # Use file to refer to the file object
    test_load = json.load(file)
test_load

{'name': 'hi', 'age': 1}

In [44]:
colums = []
for x in test_load:
    if type(test_load[x]) == str:
        colums.append(f"\n\n  @Column()\n  @ApiModelProperty()\n  {x}: string;")
    if type(test_load[x]) == int:
        colums.append(f"\n\n  @Column()\n  @ApiModelProperty()\n  {x}: number;")
#reconstruct list to string
colums = ''.join(colums)
colums

'\n\n  @Column()\n  @ApiModelProperty()\n  name: string;\n\n  @Column()\n  @ApiModelProperty()\n  age: number;'

In [45]:
entity = '''
import { Entity, PrimaryGeneratedColumn, Column } from 'typeorm';
import { ApiModelProperty } from '@nestjs/swagger';
@Entity()
export class %s {
  @PrimaryGeneratedColumn()
  @ApiModelProperty()
  id: number;%s
}
'''% (entity_name, colums)
with open(f'./src/modules/{name}/{name}.entity.ts', 'w') as file: # Use file to refer to the file object
    file.write(entity)
entity

314

# Import created entity to module 配置数据库

In [46]:
module = '''
import { Module } from '@nestjs/common';
import { TypeOrmModule } from '@nestjs/typeorm';
import { %s } from './%s.entity';
import { %sController } from './%s.controller';
import { %sService } from './%s.service';

@Module({
  imports: [
    TypeOrmModule.forFeature([%s])],
  controllers: [%sController],
  providers: [%sService]
})
export class %sModule {}

'''% (entity_name, name, entity_name,name, entity_name, name, entity_name, entity_name, entity_name, entity_name)

with open(f'./src/modules/{name}/{name}.module.ts', 'w') as file: # Use file to refer to the file object
    file.write(module)
module

370

# Regenerate controller with crud

In [47]:
controller = '''
import { Controller } from '@nestjs/common';
import { Crud, CrudController } from '@nestjsx/crud';
import { %s } from './%s.entity';
import { %sService } from './%s.service';
import { ApiUseTags } from '@nestjs/swagger';

@Crud({
  model: {
    type: %s,
  },
})
@Controller('%s')
@ApiUseTags('%s')
export class %sController implements CrudController<%s> {
  constructor(public service: %sService) {}
}
''' %(entity_name, name, entity_name, name, entity_name, name, entity_name, entity_name, entity_name, entity_name)
with open(f'./src/modules/{name}/{name}.controller.ts', 'w') as file: # Use file to refer to the file object
    file.write(controller)
controller

"import { Controller } from '@nestjs/common';\nimport { Crud, CrudController } from '@nestjsx/crud';\nimport { Lol } from './lol.entity';\nimport { LolService } from './lol.service';\nimport { ApiUseTags } from '@nestjs/swagger';\n\n@Crud({\n  model: {\n    type: Lol,\n  },\n})\n@Controller('lol')\n@ApiUseTags('Lol')\nexport class LolController implements CrudController<Lol> {\n  constructor(public service: LolService) {}\n}\n"

413

In [48]:
service = '''
import { Injectable } from '@nestjs/common';
import { %s } from './%s.entity';
import { InjectRepository } from '@nestjs/typeorm';
import { TypeOrmCrudService } from '@nestjsx/crud-typeorm';

@Injectable()
export class %sService extends TypeOrmCrudService<%s> {
  constructor(@InjectRepository(%s) repo) {
    super(repo);
  }
}

'''%(entity_name, name, entity_name, entity_name, entity_name)
with open(f'./src/modules/{name}/{name}.service.ts', 'w') as file: # Use file to refer to the file object
    file.write(service)
service

335